# 예측모델 만들기

## Preprocessing

### improt library

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     \ 512kB 9.7MB/s
  Running setup.py bdist_wheel for tqdm ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-5f1nm_04/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [8]:
import sklearn
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
data = pd.read_csv('gdrive/My Drive/공모전/Lpoint/데이터셋 만들기/lotte_drama.csv')
data.rename(columns = {'CLAC1_NM':'분류','CLAC2_NM':'중분류'},inplace = True)
data = pd.get_dummies(data,columns = ['분류','중분류'])
target = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score1.csv')
target2 = pd.read_csv('gdrive/My Drive/공모전/Lpoint/선호지수 만들기/score2.csv')
dataset = pd.merge(data,target, how = 'inner', on=['LABEL'])
dataset2 = pd.merge(data,target2, how = 'inner', on=['LABEL'])
dataset.shape, dataset2.shape

((4157, 75), (4157, 75))

In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4157 entries, 0 to 4156
Data columns (total 75 columns):
LABEL          4157 non-null object
PD_BUY_AM      4157 non-null int64
PD_BUY_CT      4157 non-null int64
PD_SALES       4157 non-null int64
BRA_SC         4157 non-null float64
CLAC3_SC       4157 non-null float64
F_10           4157 non-null float64
F_20           4157 non-null float64
F_30           4157 non-null float64
F_40           4157 non-null float64
F_50           4157 non-null float64
F_60           4157 non-null float64
F_70           4157 non-null float64
F_80           4157 non-null float64
M_10           4157 non-null float64
M_20           4157 non-null float64
M_30           4157 non-null float64
M_40           4157 non-null float64
M_50           4157 non-null float64
M_60           4157 non-null float64
M_70           4157 non-null float64
M_80           4157 non-null float64
NOT_SIGNED     4157 non-null float64
Desktop        4157 non-null float64
Mobile      

In [0]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

In [0]:
clf = RandomForestRegressor()

In [0]:
X, y = dataset.iloc[:,1:], dataset.SCORE

In [79]:
y.shape

(4157,)

In [78]:
k_fold = KFold(n_splits=10, shuffle=True, random_state = datetime.now().year)
X, y = dataset.iloc[:,1:], dataset.SCORE
for train, test in kf.split(X, y):
    clf.fit(X[train], y[train])
    print("[fold {0}] score: {2:.5f}".
          format(k, clf.score(X[test], y[test])))

KeyError: ignored

In [61]:
kf = KFold(n_splits=10, shuffle=True, random_state = datetime.now().year)
for train, test in kf.split(dataset):
  print("%s %s" % (train, test))

[   0    1    2 ... 4154 4155 4156] [   4   10   27   36   72   83   86   88   94  141  142  143  153  161
  168  175  181  197  199  201  223  226  235  242  244  248  260  272
  310  320  327  330  341  359  362  365  378  389  393  398  406  408
  417  418  440  454  475  477  483  509  516  517  538  542  566  581
  584  623  624  629  648  652  663  672  676  691  707  727  734  748
  767  788  806  809  814  821  836  844  866  870  872  878  879  889
  919  927  940  946  962  969  982 1023 1026 1063 1091 1092 1103 1134
 1149 1180 1192 1199 1209 1214 1226 1243 1260 1264 1290 1293 1297 1302
 1305 1315 1324 1331 1353 1360 1369 1372 1382 1387 1391 1392 1397 1404
 1405 1406 1415 1416 1418 1429 1436 1442 1452 1469 1470 1474 1476 1483
 1487 1493 1517 1521 1552 1561 1562 1563 1575 1579 1603 1622 1628 1631
 1649 1657 1684 1688 1699 1715 1729 1736 1753 1768 1774 1776 1782 1783
 1787 1802 1803 1806 1824 1834 1836 1837 1839 1856 1871 1889 1893 1897
 1900 1909 1923 1925 1936 1940 1964 1976 